In [ ]:
import utils
import loader
import graphics
import pandas as pd
import numpy as np

# Usar las trazas de Simva o las de Scorm
use_scorm = False

files_path = "./trazas/simva/"
cols_to_drop = ["stored", "id", "version", "actor.account.homePage", "authority.name", "authority.homePage", "context.contextActivities.category", "context.registration", "object.definition.name.en-US", "object.definition.description.en-US", "object.definition.type", "verb.display.en-US", "verb.id", "object.objectType", "result.response", "result.score.scaled", "result.completion", "result.success"]

if use_scorm:
	files_path = "./trazas/scorm/"
	cols_to_drop = ["verb.display.en-US", "id", "stored", "version", "actor.objectType", "actor.account.homePage", "result.success", "result.completion", "context.registration", "authority.objectType", "authority.account.homePage", "authority.account.name", "authority.name", "object.definition.description.en-US", "object.definition.name.en-US", "object.objectType", "context.contextActivities.category"]

files_extension = "json"


In [ ]:
############################
# Datos comunes
# Sacado de los JSONs
############################
all_users_df, users_individual_df_list = loader.load_all_files(files_path, files_extension, "timestamp", cols_to_drop, use_scorm)
n_users = len(users_individual_df_list)

game_starts_conditions = [("object.id", "GameStart")]
game_starts = utils.find_indices_by_conditions(all_users_df, game_starts_conditions)
game_starts = all_users_df.iloc[game_starts]
game_starts = game_starts.drop_duplicates(subset=["actor.account.name"])

gender_sexuality_combinations = [
	('male', 'heterosexual'), 
	('male', 'homosexual'), 
	('male', 'bisexual'), 
	('female', 'heterosexual'), 
	('female', 'homosexual'), 
	('female', 'bisexual')
]
demography = game_starts[["actor.account.name", "Gender", "Sexuality"]].values.tolist()

demography_info = {}
for user in demography:
	demography_info.update({user[0]: (user[1], user[2])})

In [ ]:
###########################
# ENCUESTAS
###########################
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

cols_to_drop = ["ID de respuesta", "Última página", "Lenguaje inicial", "Semilla", "Fecha de la última acción"]

surveys_path = "./encuestas/pre/"
pre = loader.load_surveys(surveys_path, files_extension, cols_to_drop)
graphics.display_df(pre, "Pretest")

surveys_path = "./encuestas/post/"
post = loader.load_surveys(surveys_path, files_extension, cols_to_drop)
graphics.display_df(post, "Postest")

# display(pre)
# display(post)


In [ ]:
import graphics

questions = ["¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego es demasiado largo]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego es aburrido]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego es fácil de jugar]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego es educativo]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego me hace reflexionar]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [El juego es fácil de usar en clase con los alumnos]", "¿Cómo de acuerdo estás con las siguientes afirmaciones? [Los videojuegos en general pueden ser herramientas educativas]"]

for question in questions:
	df = pd.Series(post[question]).value_counts()
	df = df.sort_index()
	display(df)
	graphics.display_pie_chart(df.values, df.index, "lol")
	# graphics.display_bar_chart(df, "lol", "cantidad", "pregunta", "blue")

In [ ]:
# Preguntas de opinion

question = "¿Qué opinas del juego?"
print(f"{question}:")
print(post[question].unique())

question = "¿Crees que has aprendido algo?"
print(f"{question}:")
print(post[question].unique())

question = "¿Qué no te ha gustado del juego y qué piensas que se podría cambiar?"
print(f"{question}:")
print(post[question].unique())

In [ ]:
def survey_comparative(pre, post):
	def extraer_num(x):
		try:
			return int(str(x).split('-')[0].strip())
		except Exception:
			return x  # Si no es del formato esperado, lo deja igual

	cols_modificar = pre.columns[4:]
	datos1 = pre.copy()
	datos2 = post.copy()

	# Aplica la extracción solo a las columnas deseadas
	datos1[cols_modificar] = datos1[cols_modificar].map(extraer_num)
	datos2[cols_modificar] = datos2[cols_modificar].map(extraer_num)

	for index, (idx1, row1) in enumerate(datos1.iterrows()):
		codigo = row1["Código de acceso"]
		fila2 = datos2[datos2["Código de acceso"] == codigo]
		y1 = row1[cols_modificar].values
		x = range(1, len(cols_modificar) + 1)  # Eje X: número de pregunta
		title = f'Comparación persona ({codigo})'
		graphics.display_line_graph(x, y1, fila2, cols_modificar, title, "Número de pregunta", "Peligrosidad")

survey_comparative(pre, post)

def survey_difference(pre, post):
	def extraer_num(x):
		try:
			return int(str(x).split('-')[0].strip())
		except Exception:
			return -1
	
	cols_modificar = pre.columns[4:]
	datos1 = pre.copy()
	datos2 = post.copy()

	datos1[cols_modificar] = datos1[cols_modificar].map(extraer_num)
	datos2[cols_modificar] = datos2[cols_modificar].map(extraer_num)

	diferencias = []

	for idx1, row1 in datos1.iterrows():
		codigo = row1["Código de acceso"]
		fila2 = datos2[datos2["Código de acceso"] == codigo]
		y1 = row1[cols_modificar].values
		if not fila2.empty:
			y2 = fila2.iloc[0][cols_modificar].values
			if not (-1 in y1 or -1 in y2):
				diff = y2 - y1
				diferencias.append(diff)

	# Convierte la lista en DataFrame para graficar fácilmente
	diferencias_df = pd.DataFrame(diferencias, columns=cols_modificar)

	x = np.arange(1, len(cols_modificar) + 1)
	promedio = diferencias_df.mean()

	graphics.display_bar(x, promedio, "Diferencia promedio (Post - Pre) por pregunta",
						"Número de pregunta", "Diferencia promedio")

survey_difference(pre, post)